Release VERSION 230109

<h1>RSoXS plan simulator</h1>
this is a full stack of the SST scan system, from the loader through to the low level acquisition code, all simulated here
run all the hidden cells first!

In [15]:
%load_ext autoreload

from rsoxs_scans.spreadsheets import load_samplesxlsx, save_samplesxlsx
from rsoxs_scans.acquisition import dryrun_bar, get_acq_details
import pathlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# userPath = pathlib.Path('/nsls2/users/alevin')
projectPath = pathlib.Path('/Users/andrew/Library/CloudStorage/OneDrive-UCB-O365/research/beamline_stuff/BNL-NSLSII/SST1/')
sheetsPath = projectPath.joinpath('2024-2')
sheetsPath.exists()

True

In [18]:
version = 'v1'
# sheetPath = sheetsPath.joinpath(f'levin_sample_bar_2024C2_{version}.xlsx')
sheetPath = sheetsPath.joinpath('out_2024-07-13_11-27-13_SpotsPickedFromImage.xlsx')
sheetPath.exists()

True

In [19]:
# load the example bar
bar = load_samplesxlsx(sheetPath)

spreadsheet version is 2023-2 Version 1.0
Reading sheet Acquisitions


In [20]:
# dry run it
outputs = dryrun_bar(bar,['apriority'],group='all')

___________________________________________________________________
Acquisition # 0 from Group: 
	Sample Name: PSS
	Sample id: TEY1
	Project: TEY_NEXAFS

Summary: load TEY1 with config WAXSNEXAFS, run nexafs with priority(Sample: 0.0, Acquisition: 1)
	Starts @ 0:00:00 takes 0:16:43 (+120 seconds for configuration change)

> Step 0: load configuration WAXSNEXAFS
> Step 1: load sample PSS
> Step 2: set Diode range to high
> Step 3: calculating a lab-frame polarization of 0.0 from the sample_frame polarization 
  input of 20 and a sample angle 20.0
> Step 4: setting sample angle to 20.0
 setting polarization to 0.0
 setting grating to rsoxs
 fly nexafs scanning from 250.0 eV to 350.0 eV on the rsoxs l/mm grating
    at speeds from 0.15 to 0.75 ev/second
> Step 5: calculating a lab-frame polarization of 52.382487859339335 from the sample_frame polarization 
  input of 55 and a sample angle 20.0
> Step 6: setting sample angle to 20.0
 setting polarization to 52.382487859339335
 setting grat

In [ ]:
(10 * 3 * 24 * 2)/60

In [ ]:
# Get more details on a given acquisition
get_acq_details(40,outputs)

In [ ]:
# get the set of all actions
actions = set()
for acq in outputs:
    for step in acq['steps']:
        actions.add(step['action'])
print(actions)

In [ ]:
arguments = {}
for action in actions:
    keys = set()
    for acq in outputs:
        for step in acq['steps']:
            if step['action'] == action:
                if 'kwargs' in step:
                    for kwarg in step['kwargs']:
                        keys.add(kwarg)
    arguments[action]=keys
    print("{:<18} : {}".format(action,keys))

<h1>Examples of edges, ratios, frames

In [ ]:
# totally manual simple scan
from rsoxs_scans.constructor import get_energies
energies = get_energies((1850,1930),100)

In [ ]:
# precise energy list
# you can fool the algorithm if you put 1 as the number of frames it will only take one frame at each of the thresholds, including the last one, so you can sort of treat this as just an energy list
# as a corollary, you can't have a total number of frames less than the thresholds that you give it in edges (it will at a minimum always take the threshold values)
energies = get_energies((250,250,250,340,340,341,280,281.45,500,500),1)

In [ ]:
# use a default
energies = get_energies('Nitrogen')

In [ ]:
# use a default with a different name
energies = get_energies('c')

In [ ]:
# use a different built in interval set
energies = get_energies('c','full','carbon nonaromatic')

In [ ]:
# get a very short version of this scan
energies = get_energies('c','very short','carbon nonaromatic')

In [ ]:
# I don't have time for this, I only want ~10 energies
energies = get_energies('c',10,'carbon nonaromatic')

In [ ]:
# also, I made single entry energies also work
energies = get_energies([350, 305, 292, 287, 282, 270, 250],'short',[5, 1, 0.2, 0.1, 1, 5])

In [ ]:
# also option to have frames as a list of exact numbers needed in each region
energies = get_energies('c',[2,5,10,40,5,2])

In [ ]:
# putting in zero effectively removes that region
energies = get_energies('c',[2,5,10,0,5,2])

In [ ]:
from rsoxs_scans.constructor import construct_exposure_times
times,_ = construct_exposure_times(get_energies([270,280,290,400],500,[2,1,10]),1)
len(times)

In [ ]:
from rsoxs_scans.constructor import get_nexafs_scan_params
get_nexafs_scan_params('n','normal')

<h1>Example of complex Exposure times

In [ ]:
construct_exposure_times(get_energies((1850,1930),100),[2,("between",1870,1900.0),4,("greater_than",1920.0),1],80)

In [ ]:
isinstance('asdf',str)